In [2]:
import os, threading, time
import time
from flask import Flask, redirect, render_template
from flask import request, make_response, jsonify
import pyttsx3
import speech_recognition as sr
from konlpy.tag import *
import warnings
from flask import Flask
from flask_cors import CORS
import pymysql
import pymysql.cursors
import socket
from ckonlpy.tag import Twitter
import gensim
import cv2
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
nltk.download('punkt')

############################## DB & MAPPING ##############################
####### DB #######
conn = pymysql.connect(program_name="mysql",
                       host="project-db-cgi.smhrd.com",
                       port=3307,
                       #charset="EUC-kR",
                       db="cgi_2_230701_4",
                       user="cgi_2_230701_4",
                       database="cgi_2_230701_4",
                       password="smhrd4",
                       cursorclass=pymysql.cursors.DictCursor)
result = []
with conn:
    #print("yap")
    #print(conn)
    with conn.cursor() as cursor:
        sql = "select menu_name from t_menu"
        cursor.execute(sql)
        result = cursor.fetchall()
        #print(result)

####### MAPPING #######
menu = []
for i in range(len(result)):
    mn = result[i]['menu_name']
    split = mn.split('(')
    menu.append(split[0])
dupl = []
for i in menu:
    if i not in dupl:
        dupl.append(i)
dictionary={'menu_name':[],
            'amount':['한','한잔','1','1잔','하나','두','두잔''2','2잔','둘','세','세잔','3','3잔','네','네잔','4','4잔','다섯','다섯잔','5','5잔','여섯','여섯잔','6','6잔','일곱','7','여덟','8','아홉','9','열','10'],
            'temperature':['뜨거운','차가운','따뜻하게','차갑게','시원한','시원하게','시원한거','뜨겁게','따뜻한','따뜻한거','핫','아이스'],
            'temperature_c':['차갑','시원','아이스','차갑게','차가운','시원한','시원하게'],
            'temperature_h':['뜨겁','따뜻','핫','뜨거운','뜨겁게','따뜻한','따뜻하게'],
            'category' : ['추천메뉴','커피','차','에이드'],
            'category_seq' : [1,2,3,4,5]
           }
temp = {'cold':['라임모히또','레몬에이드','블루레몬에이드','자몽에이드','청포도에이드','유니콘매직에이드','체리콕','메가에이드','젤라또아포가토'],
       'hot':['에스프레소','에스프레소도피오','에소프레소피에노']}
for i in range(len(dupl)):
    dictionary['menu_name'].append(dupl[i])
amount_dict = {
    '한': 1, '한잔': 1,
    '1': 1, '1잔' : 1,
    '2잔': 2, '두잔': 2,
    '2': 2, '두' : 2,
    '세': 3, '세잔': 3,
    '3': 3, '3잔' : 3,
    '네': 4, '네잔': 4,
    '4': 4, '4잔': 4,
    '다섯': 5, '다섯잔': 5,
    '5': 5, '5잔' : 5,
    '여섯': 6, '여섯잔': 6,
    '6': 6, '6잔' : 6, 
    '일곱': 7, '7': 7, '일곱잔': 7, '7잔' : 7,
    '여덟': 8, '8': 8, '여덟잔': 8, '8잔' : 8,
    '아홉': 9, '9': 9, '아홉잔': 9, '9잔' : 9,
    '열': 10, '10': 10, '열잔': 10 ,'10잔' : 10,
    '한개':1, '두개':2, '세개':3, '네개':4, '다섯개':5,
    '여섯개':6, '일곱개':7, '여덞개':8, '아홉개':9, '열개':10,
    '하나':1, '둘':2, '셋':3, '넷':4, '다섯':5, '여섯':6, '일곱':7, '여덟':8, '아홉':9, '열':10
}
num_to_hangul = {
    1:'한', 2:'두', 3:'세', 4:'네', 5:'다섯', 6:'여섯', 7:'일곱', 8:'여덟', 9:'아홉', 10:'열'
}
####### VOICE DEFINITION #######
NUM_DIC = {1:'한', 2:'두', 3:'세', 4:'네', 5:'다섯', 6:'여섯', 7:'일곱', 8:'여덟', 9:'아홉', 10:'열'}
NUMBERS_POOL = [1,2,3,4,5,6,7,8,9,10]
menus_list = []
for elem in result:
    menus_list.append(('차가운 ' if elem['menu_name'].find('ICE')!=-1 else '뜨거운 ') + elem['menu_name'].split('(')[0])
#--------------------- generate cartesian products------------------------
### intents : 메뉴/분류/시작 / 음성 버튼 / 주문 확인 / 상세 선택 / 주문내역 확인 / 카드 결제 / 결제 완료
start_responses = [ '주문이 어렵다면 음성 버튼을 누르고 말씀으로 해보세요!', '주문이 어려우시다면 음성인식 기능을 사용해보세요!']
start_inputs = ['시작' for _ in start_responses]
start_intents = ['시작' for _ in start_responses]
category_combinations = [cat+str(num) for cat in dictionary['category'] for num in NUMBERS_POOL]
category_intents = ['category' for _ in category_combinations]
category_responses = [("어떤 "+cat+"로 드릴까요?") for cat in dictionary['category'] for _ in NUMBERS_POOL]
#menu_combinations = [menu+str(number) for menu in menus_list for number in NUMBERS_POOL]
#menu_intents = ['menu' for _ in menu_combinations]
#menu_responses = [(menu+" "+NUM_DIC[num]+" 잔 드릴까요?") for menu in menus_list for num in NUMBERS_POOL]
# ordercheck_responses = ['주문 내역을 확인해주세요', '주문하신 메뉴가 맞는지 확인해주세요']
# ordercheck_inputs = ['주문내역확인' for _ in ordercheck_responses]
# ordercheck_intents = ['주문내역확인' for _ in ordercheck_responses]
options_responses = ['옵션 선택이 어려우시면 기본으로 선택하기를 눌러주세요']
options_inputs=  ['상세선택' for _ in options_responses]
options_intents = ['상세선택' for _ in options_responses]
paymethod_responses = ['결제수단을 선택해주세요']
paymethod_inputs = ['결제수단' for _ in paymethod_responses]
paymethod_intents = ['결제수단' for _ in paymethod_responses]
done_responses = ['결제가 완료되었습니다.', '영수증과 번호표를 받아가세요!']
done_inputs = ['결제완료' for _ in done_responses]
done_intents = ['결제완료' for _ in done_responses]
error_responses = ['다시 한번 말씀해주세요']
error_inputs = ['인식불가' for _ in error_responses]
error_intents = ['인식불가' for _ in error_responses]
ambig_inputs = [elem['menu_name'].split('(')[0] for elem in result]
ambig_responses = ['수량과 온도는 어떻게 하시겠습니까' for _ in ambig_inputs]
ambig_intents = ['불특정' for _ in ambig_inputs]
cancel_responses = [elem['menu_name'].split('(')[0] + ' 취소' for elem in result]
cancel_inputs = [elem['menu_name'].split('(')[0] + ' 취소' for elem in result]
cancel_intents = ['취소' for _ in cancel_responses]
dataset = pd.DataFrame(columns=['intent', 'input', 'resp'])
#####--------tweak this to add intent----------------##########
dat = {'intent':category_intents+ start_intents+paymethod_intents+done_intents+error_intents+ambig_intents+cancel_intents+options_intents,
       'input':category_combinations+ start_inputs+paymethod_inputs+done_inputs+error_inputs+ambig_inputs+cancel_inputs+options_inputs,
       'resp':category_responses+ start_responses+paymethod_responses+done_responses+error_responses+ambig_responses+cancel_responses+options_responses}
dataset = pd.concat([dataset, pd.DataFrame(dat)], ignore_index=True)

############################## METHODS & CONSTANTS ##############################
####### CONSTANTS, MODEL #######
FRAMES_TO_TAKE = 3
NOISE_THRESHOLD = 4000
CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
FILEPATH='d:/rejuvenator_champV10.h5'
MODEL = load_model(filepath=FILEPATH)
MODEL.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

####### cosine distance count vectorizer method
def cdcvm(sentence1, sentence2):
  all_sentences = [sentence1, sentence2]

  from sklearn.feature_extraction.text import CountVectorizer
  from scipy.spatial import distance

  vectorizer = CountVectorizer()
  all_sentences_to_vector = vectorizer.fit_transform(all_sentences)
  text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
  text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()

  # distance of similarity
  cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
  return round((1-cosine), 2)

#resp
def respond(text):
  maximum = float('-inf')
  response = ''
  closest = ''
  for i in dataset.iterrows():
    sim = cdcvm(text, i[1]['input'])
    if sim > maximum:
      maximum = sim
      response = i[1]['resp']
      closest = i[1]['input']
  convert_to_speech(response)
  return response

####### TEXT PROCESSOR-----------------------------------------------------------
def handle_order(tokenized_content, dictionary):
    amount = dictionary['amount']
    speech_container = {}
    m_name = ''
    menu_cnt = None
    if tokenized_content[0][1] in dictionary['menu_name']:
        m_name = tokenized_content[0][1]
    else:
        m_name = tokenized_content[0][1] + tokenized_content[0][2] 
    if m_name: # If menu name is detected
        temperature = '(ICE)' if tokenized_content[0][0] in dictionary['temperature_c'] else '(HOT)'
        for k in range(len(tokenized_content[0])):
            if tokenized_content[0][k] in amount:
                menu_cnt = amount_dict[tokenized_content[0][k]]
                break # Break the loop after finding the first amount
    if m_name and menu_cnt:
        speech_container = {"m_name": m_name +f'{temperature}', "menu_cnt": menu_cnt}
    else:
        print('Invalid order. Please try again.')
    return speech_container

####### SPEECH RECOGNITION-------------------------------------------------------
def voice_recognition_loop():
    recog = sr.Recognizer()
    mic = sr.Microphone()
    recog.energy_threshold = NOISE_THRESHOLD
    with mic as source:
        audio = recog.listen(source)
        try:
            recog.adjust_for_ambient_noise(source)
            text = recog.recognize_google(audio, language='ko-KR')
            print(text)
            content = [text]
            tokenized_content = [word_tokenize(sentence.lower()) for sentence in content]
            # Handle the order based on the detected tokens
            speech_container = handle_order(tokenized_content, dictionary)
            return speech_container
        except sr.UnknownValueError:
            print('Speech Unrecognized')
            return {"error": "Speech Unrecognized"}
        except sr.RequestError:
            print('Request Error')
            return {"error": "Request Error"}

####### TTS----------------------------------------------------------------------
def convert_to_speech(text, voice_id='HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\SPEECH\Voices\Tokens\TTS_MS_KO-KR_HEAMI_11.0'):
    engine= pyttsx3.init(driverName='sapi5')
    engine.setProperty('voice', voice_id)
    engine.setProperty('rate', 165)
    engine.say(text)
    engine.runAndWait() # might want to get this out of this scope later...we'll test it out
    engine.stop()

############################## URL MAPPING, STARTUP ##############################
app = Flask(__name__)
CORS(app, methods="*", origins="*")
#CORS(app, methods="*", resources={r"/look": {"origins": "http://localhost:8081"}, r"/listen":{"origins":"http://localhost:8081"}})
####### STT-----------------------------------------------------------------------
@app.route('/listen', methods=['GET', 'POST'])
def acquiesce():
    #********************TODO: ADD CONTEXT *****************************
    #rsps = {"convContext":resultContext, "answer":"32", "options":"basic"}
    result = voice_recognition_loop()
    return jsonify(result)

####### TTS----------------------------------------------------------------------
@app.route('/speak')
def bad_mouth():
    context = request.args.get('context')
    respond(context)  # 비동기 함수를 실행하기 위해 asyncio.ensure_future 사용
    return "EOT"

####### FACE----------------------------------------------------------------------
@app.route('/look')
def be_honest():
  #### GET FEED
  feed = cv2.VideoCapture(0)
  pics_taken = []
  while feed.isOpened():
    ret, frame = feed.read()
    if not ret:
      print("Failed to capture frame.")
      break
    #### JUDGE FRONTAL AND I DON'T MEAN YOUR FINE FRONT SIDE AND YES I DID GIGGLE
    #img = cv2.imread("*.jpg") # test
    #### INITIALIZE THE CASCADE CONSTANT FROM THE ABOVE CELL
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = CASCADE.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    is_frontal = False
    if len(faces) > 0:
      is_frontal = True
    if is_frontal:
      print("FRONTAL!")
      cropped_face =[]
      for (x, y, w, h) in faces:
        cropped_face = frame[y:y+h, x:x+w]
        print("CROPPED!")
      cropped_and_resized = cv2.resize(cropped_face, (115, 115))
      final_product = np.array([cropped_and_resized])
      print("RESIZED!")
      pics_taken.append(final_product)
      feed.release()
      break
    else:
      print('FACE FRONT ', end="")
  #### PREDICT
  predictions = []
  for pic in pics_taken:
    the_hoax = MODEL.predict(pic[:1]/255)
    try:
      predictions.append(the_hoax)
    except:
      print("Can't predict.")
  #### SAVE LANDMARKS VIA MYSQL-CONNECTOR-PYTHON (1st time only, update every year) TO T_CUST
  #### RESPONSE
  result = predictions[0].argmax()
  if result == 0 :
    print("Millenial-Z")
    return "10"
  elif result == 1:
    print("X-Millenial")
    return "30"
  elif result == 2:
    print("Boomer-X")
    return "50"

if __name__ == '__main__':
    print("☕Coffee butler at your service!☕")
    print("😁/listen, /look😙")
    app.run()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smhrd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


☕Coffee butler at your service!☕
😁/listen, /look😙
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-07-31 11:14:26,301] ERROR in app: Exception on /speak [GET]
Traceback (most recent call last):
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c